In [1]:
import numpy as np
from python.py_rle import mask2rle, rle2mask 
from cython_rle import cy_mask_to_rle
from py_multi_backend import (
    arr2rle, 
    c_order_mask2rle, 
    f_order_mask2rle, 
    thread_arr2rle, 
    thread_f_order_mask2rle, 
    thread_c_order_mask2rle,
    rle2mask as rust_rle2mask
)
from pycocotools import mask as cocomask

In [2]:
SIZE = (3072, 3072)
rand_mask = np.random.random(SIZE) < 0.5
static_mask = np.ones(SIZE) < 0.5
rand_rle = arr2rle(rand_mask.ravel("F"))
static_rle = arr2rle(static_mask.ravel("F"))

static_seg_rle = cocomask.encode(np.asfortranarray(static_mask))
static_seg_rle["counts"] = static_seg_rle["counts"].decode("utf-8")

rand_seg_rle = cocomask.encode(np.asfortranarray(rand_mask))
rand_seg_rle["counts"] = rand_seg_rle["counts"].decode("utf-8")

### RLE to Mask Benchmarks

In [3]:
assert (rust_rle2mask(static_rle, SIZE) == static_mask).all()
assert (rust_rle2mask(rand_rle, SIZE) == rand_mask).all()

In [4]:
%%timeit
_ = rust_rle2mask(static_rle, SIZE)

1.42 ms ± 60.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
%%timeit
_ = rust_rle2mask(rand_rle, SIZE)

195 ms ± 390 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
_ = rle2mask(static_seg_rle)

12.5 ms ± 256 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
_ = rle2mask(rand_seg_rle)

97.5 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Mask to RLE Benchmarks

In [ ]:
assert cy_mask_to_rle(rand_mask)["counts"] == mask2rle(rand_mask)["counts"]
assert arr2rle(rand_mask.ravel(order="F")) == mask2rle(rand_mask)["counts"]
assert f_order_mask2rle(rand_mask) == mask2rle(rand_mask)["counts"]

In [ ]:
%%timeit
_ = arr2rle(rand_mask.ravel("F"))

In [ ]:
%%timeit
_ = arr2rle(static_mask.ravel("F"))

In [ ]:
%%timeit
_ = f_order_mask2rle(rand_mask)

In [ ]:
%%timeit
_ = f_order_mask2rle(static_mask)

In [ ]:
%%timeit
_ = c_order_mask2rle(rand_mask)

In [ ]:
%%timeit
_ = c_order_mask2rle(static_mask)

In [ ]:
%%timeit
_ = cy_mask_to_rle(rand_mask)

In [ ]:
%%timeit
_ = cy_mask_to_rle(static_mask)

In [ ]:
%%timeit
_ = mask2rle(rand_mask)

In [ ]:
%%timeit
_ = mask2rle(static_mask)